In [1]:
#!pip install azure-storage-blob

In [2]:
#import os

#print("AZURE_CLIENT_ID:", os.getenv("AZURE_CLIENT_ID"))
#print("AZURE_TENANT_ID:", os.getenv("AZURE_TENANT_ID"))
#print("AZURE_CLIENT_SECRET:", os.getenv("AZURE_CLIENT_SECRET"))


In [1]:
from azure.identity import AzureCliCredential
from azure.keyvault.secrets import SecretClient

# Azure Key Vault configuration
key_vault_name = "***"
secret_name = "***"

key_vault_uri = f"https://{key_vault_name}.vault.azure.net"

# Create a SecretClient using AzureCliCredential
credential = AzureCliCredential()
client = SecretClient(vault_url=key_vault_uri, credential=credential)

# Retrieve the API key from Azure Key Vault
api_key = client.get_secret(secret_name).value

#print(f"The API key is: {api_key}")

# Azure Key Vault configuration
key_vault_name2 = "***"
secret_name2 = "***"

key_vault_uri2 = f"https://{key_vault_name2}.vault.azure.net"

# Create a SecretClient using AzureCliCredential
credential2 = AzureCliCredential()
client2 = SecretClient(vault_url=key_vault_uri2, credential=credential2)

# Retrieve the API key from Azure Key Vault
api_key2 = client2.get_secret(secret_name2).value

In [2]:
from azure.storage.blob import BlobServiceClient

blob_service_client = BlobServiceClient.from_connection_string(api_key2)
container_client = blob_service_client.get_container_client("container1\weatherdata")

In [4]:
import os
import requests
import json
import time

from kafka import KafkaProducer
from kafka import KafkaConsumer

# Kafka configuration
bootstrap_servers='192.168.1.88:9092'
topic_name = 'weatherdata'

producer = KafkaProducer(bootstrap_servers=bootstrap_servers, value_serializer=lambda v: json.dumps(v).encode('utf-8'))


# Variables for latitude and longitude
lat = 69.0  # Example latitude
lon = 69.0  # Example longitude



# Function to fetch weather data
def fetch_weather_data():
    url = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&appid={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

# Function to send data to Kafka
def send_to_kafka(data):
    producer.send('weatherdata', value=data)
    producer.flush()

# Main loop to fetch and send data periodically
def main():
    start_time = time.time()  # Record the start time
    duration = 2 * 60  # Run for 2 minutes (120 seconds)
    
    while time.time() - start_time < duration:
        weather_data = fetch_weather_data()  # Fetch the weather data every 30 seconds
        if weather_data:
            send_to_kafka(weather_data)  # Send the data to Kafka
        time.sleep(30)  # Wait for 30 seconds before fetching again  



# Function to consume from Kafka and upload to Azure Blob
def consume_and_upload():
    consumer = KafkaConsumer(topic_name, bootstrap_servers=bootstrap_servers, 
                             value_deserializer=lambda v: json.loads(v.decode('utf-8')))
    for message in consumer:
        data = message.value
        blob_name = f"data_batch_{int(time.time())}.json"
        blob_client = container_client.get_blob_client(blob_name)
        try:
            blob_client.upload_blob(json.dumps(data))
            print(f"Uploaded blob: {blob_name}")
        except Exception as e:
            print(f"Failed to upload blob: {e}")



if __name__ == "__main__":
    # Start the consumer in a separate thread
    import threading
    consumer_thread = threading.Thread(target=consume_and_upload)
    consumer_thread.start()

    # Run the main function
    main()

Uploaded blob: data_batch_1727110961.json
Uploaded blob: data_batch_1727110991.json
Uploaded blob: data_batch_1727111022.json
Uploaded blob: data_batch_1727111052.json


In [ ]:
#from geosky import geo_plug

#len(geo_plug.all_CountryNames())

#geo_plug.all_Country_StateNames()

#geo_plug.all_State_CityNames('North Carolina')# name == 'all' or state name
#geo_plug.all_State_CityNames('Odisha')